In [14]:
import os
import requests
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import time
import math
import numpy as np
from datetime import date, datetime, timedelta

#### Obtendo token de acesso 

In [15]:
def token_login():
    url = "https://api.maplink.global/oauth/client_credential/accesstoken?grant_type=client_credentials"

    headers={"Content-Type": "application/x-www-form-urlencoded",
            }

    body = {"client_id": "",
            "client_secret": ""}


    response = requests.request("POST", url, headers=headers, data=body)

    token = response.json()["access_token"]

    return token

#### Timestamps 

In [16]:
today = date.today()

# alt = today - timedelta(7)

data = today

init_time0 = datetime.strptime("00:00:00", "%H:%M:%S").time()

init_time0 = int(datetime.timestamp(datetime.combine(data, init_time0))*1000)

init_time = datetime.strptime("00:00:00", "%H:%M:%S").time()

init_time = int(datetime.timestamp(datetime.combine(data, init_time))*1000)

end_time = datetime.strptime("23:59:00", "%H:%M:%S").time()

end_time = int(datetime.timestamp(datetime.combine(data, end_time))*1000)

init_time2 = datetime.strptime("00:00:00", "%H:%M:%S").time()

init_time2 = int(datetime.timestamp(datetime.combine(data, init_time2))*1000)

end_time2 = datetime.strptime("23:59:00", "%H:%M:%S").time()

end_time2 = int(datetime.timestamp(datetime.combine(data, end_time2))*1000)

# init_time0 = 1513750219000

# init_time = 1513753200000

# end_time = 1513796400000

# init_time2 = 1513756800000

# end_time2 = 1513771200000

print(init_time0, init_time, end_time, init_time2, end_time2)

1672369200000 1672369200000 1672455540000 1672369200000 1672455540000


#### Corners

In [17]:
# Separa o CT, nome do corner e endereço do corner

path = r''

end_corners = pd.read_excel(path)
end_corners.reset_index

#end_corners['END_CORNER'] = end_corners['ENDEREÇO DO CORNER '] + " - " + end_corners['CIDADE'] + " - " + end_corners['UF'] + ", " + end_corners['CEP']
end_corners['END_CORNER'] = end_corners['CEP']

end_corners = end_corners[['CT', 'CORNER', 'END_CORNER']]

end_corners = end_corners.rename(columns={'ENDEREÇO DO CORNER': 'END_CORNER'})

end_corners.head()

,CT,CORNER,END_CORNER
0,CT002156,AG 3931 IPATINGA MG,35162-359
1,CT000299,AG 4036 JABOATAO GUARARAPES PE,54335-160
2,CT000247,AG 4168 SINOP MT,78550-226
3,CT001296,AG 4343 BELEM PA,66013-060
4,CT000279,AG. 0100_TERESINA_PI,64000-060


#### Oppay

In [18]:
#Puxa os dados do Oppay

path = r''

os_oppay = pd.read_csv(path ,sep=';')

#Organiza os dados de endereço 
#os_oppay['END_EC'] = os_oppay['Endereço'] + " - " + os_oppay['Cidade'] + " - " + os_oppay['Estado'] + ", " + os_oppay['CEP']

os_oppay['END_EC'] = os_oppay['CEP']

os_filter = os_oppay[['Numero Referencia', 'END_EC', 'Serviço', 'Centro Trabalho','Data Abertura']]

os_filter = os_filter.rename(columns={'Centro Trabalho': 'CT'})

os_data = pd.merge(os_filter, end_corners, on='CT', how='inner')

os_data = os_data[0:124] #slicing de linhas para teste

# os_data = os_data.loc[(os_data['CORNER'] == "AG.0010_CAMPINAS_SP_CORNER")] #slicing de corner

os_data.tail(8)

,Numero Referencia,END_EC,Serviço,CT,Data Abertura,CORNER,END_CORNER
116,57474102,60035-161,INSTALACAO,CT001045,22/08/2022 15:54,AG.3132_FORTALEZA_CE_CORNER,60025-130
117,57474121,60835-775,INSTALACAO,CT001045,22/08/2022 15:55,AG.3132_FORTALEZA_CE_CORNER,60025-130
118,57474853,60060-570,MANUTENCAO,CT001045,22/08/2022 16:41,AG.3132_FORTALEZA_CE_CORNER,60025-130
119,57475920,60165-078,INSTALACAO,CT001045,22/08/2022 18:31,AG.3132_FORTALEZA_CE_CORNER,60025-130
120,57475921,60165-078,INSTALACAO,CT001045,22/08/2022 18:31,AG.3132_FORTALEZA_CE_CORNER,60025-130
121,57475986,60170-320,INSTALACAO,CT001045,22/08/2022 18:42,AG.3132_FORTALEZA_CE_CORNER,60025-130
122,57476120,60035-161,TROCA CHIP VENDA,CT001045,22/08/2022 19:37,AG.3132_FORTALEZA_CE_CORNER,60025-130
123,57475597,60351-552,INSTALACAO,CT001045,22/08/2022 17:43,AG.3132_FORTALEZA_CE_CORNER,60025-130


In [19]:
os_data['Numero Referencia'] = os_data['Numero Referencia'].astype('string')

df_group1 = os_data.groupby(by = [os_data['CORNER'], os_data['END_CORNER'], os_data['END_EC'],]).agg(', '.join)

#df_group.sort_values(by='Tempo de Deslocamento (min)', ascending=True, inplace=True)

df_group1.reset_index(inplace=True)

df_group1.head(8)

,CORNER,END_CORNER,END_EC,Numero Referencia,Serviço,CT,Data Abertura
0,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-030,57472215,MANUTENCAO,CT001045,22/08/2022 13:53
1,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-061,57469166,INSTALACAO,CT001045,22/08/2022 11:08
2,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-240,57466887,DESINSTALACAO,CT001045,22/08/2022 09:05
3,AG.3132_FORTALEZA_CE_CORNER,60025-130,60030-030,57473561,DESINSTALACAO,CT001045,22/08/2022 15:25
4,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-100,57472031,MANUTENCAO,CT001045,22/08/2022 13:40
5,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-130,57471102,INSTALACAO,CT001045,22/08/2022 12:13
6,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-161,"57474102, 57476120","INSTALACAO, TROCA CHIP VENDA","CT001045, CT001045","22/08/2022 15:54, 22/08/2022 19:37"
7,AG.3132_FORTALEZA_CE_CORNER,60025-130,60050-130,57473375,MANUTENCAO,CT001045,22/08/2022 15:13


In [20]:
cep_list_ec = df_group1['END_EC']
cep_list_corner = df_group1['END_CORNER']

#### Geocode ECs

In [21]:
token = token_login()

zip_list = cep_list_ec

df_group1['lat'] = np.zeros(len(df_group1['END_EC']))
df_group1['lon'] = np.zeros(len(df_group1['END_EC']))
df_group1['label'] = np.zeros(len(df_group1['END_EC']))
aux =0

for zipcode in zip_list:
    
    print("EC ", aux, " ", zipcode, "\n")
    
    url = "https://api.maplink.global/geocode/v1/geocode"

    headers={"Content-Type": "application/json",
             "Authorization": "Bearer " + token_login()
            }

    body = json.dumps({"zipcode": zipcode})

    response = requests.request("POST", url, headers=headers, data=body)
    
    df_group1['lat'][aux] = response.json()['results'][0]['address']['mainLocation']['lat']
    df_group1['lon'][aux] = response.json()['results'][0]['address']['mainLocation']['lon']
    df_group1['label'][aux] = response.json()['results'][0]['label']
    aux = aux+1

EC  0   60020-030 



C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\3059769088.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group1['lat'][aux] = response.json()['results'][0]['address']['mainLocation']['lat']
C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\3059769088.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group1['lon'][aux] = response.json()['results'][0]['address']['mainLocation']['lon']
C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\3059769088.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

EC  1   60020-061 

EC  2   60020-240 

EC  3   60030-030 

EC  4   60035-100 

EC  5   60035-130 

EC  6   60035-161 

EC  7   60050-130 

EC  8   60060-120 

EC  9   60060-570 

EC  10   60110-345 

EC  11   60120-001 

EC  12   60120-385 

EC  13   60130-670 

EC  14   60150-161 

EC  15   60150-165 

EC  16   60160-230 

EC  17   60165-078 

EC  18   60165-121 

EC  19   60170-310 

EC  20   60170-320 

EC  21   60176-065 

EC  22   60180-480 

EC  23   60181-330 

EC  24   60182-292 

EC  25   60351-552 

EC  26   60440-180 

EC  27   60740-020 

EC  28   60821-716 

EC  29   60835-750 

EC  30   60835-775 

EC  31   60865-240 

EC  32   60872-690 

EC  33   61650-500 

EC  34   61700-000 

EC  35   62680-000 

EC  36   89204-041 

EC  37   89220-720 

EC  38   89225-450 

EC  39   89226-340 

EC  40   89226-684 

EC  41   89227-600 

EC  42   89227-645 

EC  43   89228-300 

EC  44   89228-740 

EC  45   89229-120 

EC  46   89230-648 

EC  47   89237-060 

EC  48   23016-510 

E

In [22]:
token = token_login()

base_corner = df_group1[['CORNER', 'END_CORNER']].drop_duplicates()

base_corner['lat_corner'] = np.zeros(len(base_corner['END_CORNER']))
base_corner['lon_corner'] = np.zeros(len(base_corner['END_CORNER']))
base_corner['label_corner'] = np.zeros(len(base_corner['END_CORNER']))

for index, row in base_corner.iterrows():

    i = 1
    print("Corner", row['CORNER'], "\n")
    i+=1
    
    url = "https://api.maplink.global/geocode/v1/geocode"

    headers={"Content-Type": "application/json",
             "Authorization": "Bearer " + token_login()
            }

    body = json.dumps({"zipcode": row['END_CORNER']})

    response = requests.request("POST", url, headers=headers, data=body)
    
    base_corner['lat_corner'][index] = response.json()['results'][0]['address']['mainLocation']['lat']
    base_corner['lon_corner'][index] = response.json()['results'][0]['address']['mainLocation']['lon']
    base_corner['label_corner'][index] = response.json()['results'][0]['label']
    
df_group1 = pd.merge(left=df_group1, right=base_corner[['CORNER', 'lat_corner', 'lon_corner', 'label_corner']], how='left', on='CORNER')

Corner AG.3132_FORTALEZA_CE_CORNER 



C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\2328957662.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_corner['lat_corner'][index] = response.json()['results'][0]['address']['mainLocation']['lat']
C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\2328957662.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_corner['lon_corner'][index] = response.json()['results'][0]['address']['mainLocation']['lon']
C:\Users\jonatas.ribeiro\AppData\Local\Temp\ipykernel_16128\2328957662.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

Corner AG.3176_JOINVILLE_SC 

Corner AG.3350_RIO DE JANEIRO_RJ 

Corner AG.3504_MONTES CLAROS_MG_CORNER 



In [23]:
df_group1.head(8)

,CORNER,END_CORNER,END_EC,Numero Referencia,Serviço,CT,Data Abertura,lat,lon,label,lat_corner,lon_corner,label_corner
0,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-030,57472215,MANUTENCAO,CT001045,22/08/2022 13:53,-3.727480,-38.531000,"Rua General Sampaio, Centro, 60020030, Fortale...",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
1,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-061,57469166,INSTALACAO,CT001045,22/08/2022 11:08,-3.747645,-38.538630,"Rua Marechal Deodoro, Benfica, 60020061, Forta...",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
2,AG.3132_FORTALEZA_CE_CORNER,60025-130,60020-240,57466887,DESINSTALACAO,CT001045,22/08/2022 09:05,-3.741775,-38.535045,"Rua Aracati, Benfica, 60020240, Fortaleza, CE",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
3,AG.3132_FORTALEZA_CE_CORNER,60025-130,60030-030,57473561,DESINSTALACAO,CT001045,22/08/2022 15:25,-3.725200,-38.524475,"Rua Sobral, Centro, 60030030, Fortaleza, CE",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
4,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-100,57472031,MANUTENCAO,CT001045,22/08/2022 13:40,-3.732360,-38.525110,"Rua Pedro I, Centro, 60035100, Fortaleza, CE",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
5,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-130,57471102,INSTALACAO,CT001045,22/08/2022 12:13,-3.734520,-38.524700,"Rua General Clarindo de Queiroz, Centro, 60035...",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
6,AG.3132_FORTALEZA_CE_CORNER,60025-130,60035-161,"57474102, 57476120","INSTALACAO, TROCA CHIP VENDA","CT001045, CT001045","22/08/2022 15:54, 22/08/2022 19:37",-3.730915,-38.538960,"Rua Meton de Alencar, Centro, 60035161, Fortal...",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."
7,AG.3132_FORTALEZA_CE_CORNER,60025-130,60050-130,57473375,MANUTENCAO,CT001045,22/08/2022 15:13,-3.747815,-38.528095,"Avenida Felino Barroso, Fátima, 60050130, Fort...",-3.729265,-38.52777,"Rua Floriano Peixoto, Centro, 60025130, Fortal..."


#### API

In [24]:
token = token_login()

num_tec = 1

tec_list = []
for i in range(1, num_tec + 1):
    tec_list.append(
        {
                    "name": "VUC_" + str(i),
                    "vehicleType": "VUC",
                    "legislationProfile": "DEFAULT",
                    "availablePeriods": [
                        {"timeWindow": {
                                "start": init_time,
                                "end": end_time
                            }
                        }
                    ]
                })


    
if not os.path.exists("Rotas"):
    os.mkdir("Rotas")

for x in range(len(base_corner)) :
    
    print("Iniciando corner: ", list(base_corner['CORNER'])[x])
    inicio = time.time()

    df_corner_rota = df_group1[df_group1['CORNER'] == list(base_corner['CORNER'])[x]]
    
    site_list = []
    op_list = []

    for index, row in df_corner_rota.iterrows():
        dic_site = {'name': row['label'], 
               'coordinates': {'latitude': row['lat'], 'longitude': row['lon']}, 
               "logisticConstraints": "DEFAULT"}
        dic_op = {
                "id": row['Numero Referencia'],
                "weight": 1,
                "volume": 1,
                "product": "DEFAULT",
                "type": "DELIVERY",
                "depotSite": row['label_corner'],
                "customerSite": row['label'],
                "customerTimeWindows": [
                    {
                        "start": init_time2,
                        "end": end_time2
                    }
                ]
            }

        site_list.append(dic_site)
        op_list.append(dic_op)

    url = "https://api.maplink.global/planning/v1/problems"

    headers={"Content-Type": "application/json",
             "Authorization": "Bearer " + token
            }

    body = json.dumps(
        {
        "optimizationProfile": "BRAZIL46",
        "tripsProfile": "MAPLINKBR",
        "startDate": init_time0,
        "legislationProfiles": [
            {
                "name": "DEFAULT"
            }
        ],
        "logisticConstraints": [
            {
                "name": "DEFAULT"
            }
        ],
        "products": [
            {
                "name": "DEFAULT"
            }
        ],
        "sites": [
            {
                "name": "Cliente1",
                "coordinates": {
                    "latitude": -23.507608,
                    "longitude": -46.587145
                },
                "logisticConstraints": "DEFAULT"
            },
            {
                "name": "Cliente2",
                "coordinates": {
                    "latitude": -23.621059,
                    "longitude": -46.756193
                },
                "logisticConstraints": "DEFAULT"
            }
        ],
        "depots": [
            {
                "name": "CD",
                "coordinates": {
                    "latitude": -23.503939,
                    "longitude": -46.498419
                },
                "logisticConstraints": "DEFAULT"
            }
        ],
        "vehicleTypes": [
            {
                "name": "VUC",
                "maxVolume": 20,
                "maxWeight": 1200,
                "size": 1
            }
        ],
        "vehicles": [
            {
                "name": "VUC_1",
                "vehicleType": "VUC",
                "legislationProfile": "DEFAULT",
                "availablePeriods": [
                    {"timeWindow": {
                            "start": init_time,
                            "end": end_time
                        }
                    }
                ]
            }
        ],
        "operations": [
            {
                "id": "P1",
                "weight": 100,
                "volume": 2,
                "product": "DEFAULT",
                "type": "DELIVERY",
                "depotSite": "CD",
                "customerSite": "Cliente1",
                "customerTimeWindows": [
                    {
                        "start": init_time2,
                        "end": end_time2
                    }
                ]
            },
            {
                "id": "P2",
                "weight": 250,
                "volume": 5,
                "product": "DEFAULT",
                "type": "DELIVERY",
                "depotSite": "CD",
                "customerSite": "Cliente2",
                "customerTimeWindows": [
                    {
                        "start": init_time2,
                        "end": end_time2
                    }
                ]
            }
        ]
    })

    dic = json.loads(body)

    depot = [{
                "name": df_corner_rota['label_corner'].iloc[0],
                "coordinates": {
                    "latitude": df_corner_rota['lat_corner'].iloc[0],
                    "longitude": df_corner_rota['lon_corner'].iloc[0]
                },
                "logisticConstraints": "DEFAULT"
            }]

    dic.update({'sites': site_list, 'operations': op_list, 'depots': depot, 'vehicles': tec_list})

    body = json.dumps(dic)

    response = requests.request("POST", url, headers=headers, data=body)
    print("Response POST problems:", response)
    resolve = response.json()

    url = "https://api.maplink.global/planning/v1/solutions/"+ resolve['id']

    headers={"Content-Type": "application/json",
             "Authorization": "Bearer " + token
            }

    time.sleep(120)

    response = requests.request("GET", url, headers=headers, data=body)
    print("Response GET solutions:", response)
    data = response.json()
#     print(json.dumps(data))

    arrival = []
    departure = []
    distance = []
    duration = []
    num_os = []
    for activity in data['vehicleRoutes'][0]['routes'][0]['activities']:
        if activity['activity'] == 'DRIVING':
            arrival.append(activity['arrivalSite'])
            departure.append(activity['departureSite'])
            distance.append("{:.2f}".format(activity['distance']/1000))
            duration.append("{:.2f}".format(activity['nominalDuration']/60))
        if activity['activity'] == 'DELIVERY':
            num_os.append(activity['operations'][0])

    df = pd.DataFrame({#'Número OS': num_os,
                       'Saída': departure[:-1], 
                       'Destino': arrival[:-1], 
                       'Distância (km)': distance[:-1], 
                       'Duração (min)': duration[:-1]})

    df.to_excel('Rotas/rota_' + str(list(base_corner['CORNER'])[x]) +'.xlsx')     
    
    fim = time.time()
    print("Tempo total: ", "{:.2f}".format(fim - inicio), "segundos\n")

Iniciando corner:  AG.3132_FORTALEZA_CE_CORNER
Response POST problems: <Response [201]>
Response GET solutions: <Response [200]>
Tempo total:  121.80 segundos

Iniciando corner:  AG.3176_JOINVILLE_SC
Response POST problems: <Response [201]>
Response GET solutions: <Response [200]>
Tempo total:  122.08 segundos

Iniciando corner:  AG.3350_RIO DE JANEIRO_RJ
Response POST problems: <Response [201]>
Response GET solutions: <Response [200]>
Tempo total:  122.01 segundos

Iniciando corner:  AG.3504_MONTES CLAROS_MG_CORNER
Response POST problems: <Response [201]>
Response GET solutions: <Response [200]>
Tempo total:  121.12 segundos

